Given input , return the balanced equation.
Eample input string: ” K Mn O 4 + H 2 O 2 = Mn O 2 + H 2 O + O 2 + K O H ”. Assume input
has spaces separating every element name and numbers as shown, also for ‘+’ and ‘=’.
Example output: ” 2 K Mn O 4 + 3 H 2 O 2 = 2 Mn O 2 + 2 H 2 O + 3 O 2 + 2 K O H ” and ” [
-2,- 3, 2, 3, 3, 2 ]”, i.e. output string for balanced chemical equation and the vector of stochiometric
coefficients with negative values for reactants.
HINT: (a) For the above reaction, atomList is ’K’, ’Mn’, ’O’, ’H’
(b) chemVectors for ‘ K Mn O 4’ is [1,1,4,0] and ’ K O H ’ is [1,0,1,1]
(c) construct matrix equation for chemical equation.
1

In [1]:
import numpy as np

In [2]:
equation = "K Mn O 4 + H 2 O 2 = Mn O 2 + O 2 + K O H"

In [3]:
import re
import numpy as np
import scipy.linalg as la

def findUniqueElements(equation):
    return np.unique(elementName.findall(equation))

elementPattern = re.compile(r'[A-Z]{1}[a-z]{0,}[0-9]*')
elementName = re.compile(r'[A-Z]{1}[a-z]{0,}')
elementNumber = re.compile(r'[0-9]{1,}')

def getColumnVector(element, elementVectors): #returns vector containing number of atoms of each element in compound
    elementVector = np.zeros(len(elementVectors))
    elements = elementPattern.findall(element)
    for element in elements:
        # print(element)
        name = elementName.match(element).group()
        number = elementNumber.search(element)
        if number:
            number = int(number.group())
        else:
            number = 1
        # print(np.where(elementVectors == name)[0][0], number)
        elementVector[np.where(elementVectors == name)[0][0]] = number
    return elementVector

elementVectors = findUniqueElements(equation)
print(elementVectors)

def getEquationMatrix(equation, elementVectors):
    equation = equation.replace(' ', '')
    reactants, products = equation.split('=')
    reactants = reactants.split('+')
    products = products.split('+')

    reactantMatrix = np.zeros((len(elementVectors), len(reactants)), dtype=int)
    productMatrix = np.zeros((len(elementVectors), len(products)), dtype=int)

    for i, reactant in enumerate(reactants):
        reactantMatrix[:, i] = getColumnVector(reactant, elementVectors)
    for i, product in enumerate(products):
        productMatrix[:, i] = getColumnVector(product, elementVectors)
    
    productMatrix *= -1  # Multiply product matrix by -1

    # Join reactant and product matrices
    equationMatrix = np.hstack((reactantMatrix, productMatrix))

    return equationMatrix

equationMatrix = getEquationMatrix(equation, elementVectors)
print(equationMatrix)

nullspace_basis = la.null_space(equationMatrix)
print(nullspace_basis)

def getBalancedEquation(equation, elementVectors, nullspace_basis):
    equation = equation.replace(' ', '')
    reactants, products = equation.split('=')
    reactants = reactants.split('+')
    products = products.split('+')

    coefficients = np.round(nullspace_basis / nullspace_basis[0])
    coefficients = coefficients.astype(int)

    reactantCoefficients = coefficients[:len(reactants)]
    productCoefficients = coefficients[len(reactants):]

    balancedEquation = ''
    for i, reactant in enumerate(reactants):
        balancedEquation += str(reactantCoefficients[i][0]) + reactant + ' + '
    balancedEquation = balancedEquation[:-3] + ' = '
    for i, product in enumerate(products):
        balancedEquation += str(productCoefficients[i][0]) + product + ' + '

    return balancedEquation[:-3]



['H' 'K' 'Mn' 'O']
[[ 0  2  0  0 -1]
 [ 1  0  0  0 -1]
 [ 1  0 -1  0  0]
 [ 4  2 -2 -2 -1]]
[[0.48507125]
 [0.24253563]
 [0.48507125]
 [0.48507125]
 [0.48507125]]
